#Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch.utils.data import Dataset 
import cv2
import numpy as np 
import pandas as pd
import os
from torchvision import transforms
import random
import numpy as np
import requests
import time
import torch.nn as nn
import sys
import math
from torch.nn import functional
import torch.utils.data as data
import json
import csv
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torchvision.models as models
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
%cd /content/drive/My Drive/CS 5242 Project/New codes

In [ ]:
#%cd /content/drive/MyDrive/Neural Networks/Project/CS 5242 Project/New codes

/content/drive/MyDrive/Neural Networks/Project/CS 5242 Project/New codes


In [ ]:
data_location='Data'
vocab_file='vocab_new.csv'
relationship_json='relationship.json'
object1_object2_json='object1_object2.json'
train_labels = 'Data/train.csv'
train_features_path = 'Data/train_cnn' # 'Data/optical_cnn/train_cnn'
test_features_path = 'Data/test_cnn'
optical_test_features_path = 'Data/optical_cnn/test_cnn'

In [ ]:
#Vocab file
vocab = pd.read_csv(data_location+'/'+vocab_file)

#Relation file
with open(os.path.join(data_location, relationship_json)) as json_file:
    relations = json.load(json_file)

#Objects file
with open(os.path.join(data_location, object1_object2_json)) as json_file:
    objects = json.load(json_file)    

In [ ]:
#Functions to map words to and from vocab
def map_index_to_word_original(index, otype):
  if otype == 'relation':
    rel_key_list = list(relations.keys())
    rel_value_list = list(relations.values())
    return rel_key_list[rel_value_list.index(index)]
  else:
    objects_key_list = list(objects.keys())
    objects_value_list = list(objects.values())
    return objects_key_list[objects_value_list.index(index)]  

def map_word_to_index_original(word, otype):
  if otype == 'relation':
    return relations[word]
  else:
    return objects[word]

def map_word_to_index(word):
  return vocab[vocab.word == word].iloc[-1]['index']

def map_index_to_word(index):
  return vocab[vocab.index == index].iloc[-1]['word']   

def map_to_original_object(index):
  word = map_index_to_word(index)
  return map_word_to_index_original(word, 'object')

def map_to_original_relation(index):
  word = map_index_to_word(index)
  return map_word_to_index_original(word, 'relation')           

#Custom DataLoader

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, transform=None, mode='train'):
        if mode == 'train':
          self.dataframe=pd.read_csv(train_labels)
        self.mode = mode

    def __len__(self):
      if self.mode == 'train':
        return len(self.dataframe)
      else:
        return 119  

    def __getitem__(self, index):
        """ get a video """
        
        if self.mode == 'train':
          self.train_features = np.load('{}/train_cnnfeatures{}.npy'.format(train_features_path, index))

          object1 = self.dataframe.iloc[index]['object1']
          object1 = map_word_to_index(map_index_to_word_original(object1, 'object'))

          relation = self.dataframe.iloc[index]['relation']
          relation = map_word_to_index(map_index_to_word_original(relation, 'relation'))

          object2 = self.dataframe.iloc[index]['object2']
          object2 = map_word_to_index(map_index_to_word_original(object2, 'object'))

          return self.train_features,object1,relation,object2
        elif self.mode =='test':
          print(index)
          self.test_features = np.load('{}/test_cnnfeatures{}.npy'.format(test_features_path, index))
          return self.test_features

        elif self.mode == 'ensemble':
          #For ensemble
          self.test_features = np.load('{}/test_cnnfeatures{}.npy'.format(test_features_path, index))
          self.optical_test_features = np.load('{}/test_cnnfeatures{}.npy'.format(optical_test_features_path, index))
          return self.test_features , self.optical_test_features


#Model Definition

In [ ]:
batch_size = 8
feature_dim = 2048         # dimensionality of cnn features
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 50            # number of training epochs (1 for testing)
print_every = 200          # determines window for printing average loss
vocab_size = len(vocab)
validation_split=0.2
is_bidirectional = True

In [ ]:
class Model(nn.Module):
    def __init__(self, feature_dim=2048, hidden_size=512):
        self.feature_dim = feature_dim
        self.hidden_size = hidden_size
        super(Model,self).__init__()
        self.lstm = nn.LSTM(input_size=self.feature_dim,
                            hidden_size=self.hidden_size,  # LSTM hidden units
                            num_layers=1,  # number of LSTM layer
                            bias=True,  # use bias weights b_ih and b_hh
                            batch_first=True,  # input & output will have batch size as 1st dimension
                            dropout=0,  # Not applying dropout
                            bidirectional=is_bidirectional,  # unidirectional LSTM
                            )
        self.lstm_cell = nn.LSTMCell(input_size=vocab_size,
                            hidden_size=self.hidden_size,  # LSTM hidden units
                            bias=True,  # use bias weights b_ih and b_hh
                            )
        # self.prelu = nn.PReLU()
        self.dropout = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.linear = nn.Linear(self.hidden_size, vocab_size)
        # self.ln = nn.LayerNorm(self.hidden_size, elementwise_affine=False)

    def forward(self, images, object1=None, relation=None):
        # print('images shape',images.shape)
        hidden_state = torch.zeros((2, images.shape[0], self.hidden_size)).cuda()
        cell_state = torch.zeros((2, images.shape[0], self.hidden_size)).cuda()
        
        features = images.squeeze(-1).squeeze(-1) #feature shape: batch, 30, 2048, 1, 1
        # features = self.prelu(features)
        output, (hidden_state, cell_state) = self.lstm(features, (hidden_state, cell_state))
        # output, (hidden_state, cell_state) = self.lstm(features)

        #2, 1, 512 (num of layers * num of directions, batch, dimension)
        print(hidden_state.shape)


        hidden_state = self.dropout(hidden_state)
        cell_state = self.dropout(cell_state)
        
        if is_bidirectional:
          # Sum
          hidden_state = torch.sum(torch.stack([hidden_state[0], hidden_state[1]]), 0).unsqueeze(0)
          cell_state = torch.sum(torch.stack([cell_state[0], cell_state[1]]), 0).unsqueeze(0)

          # Concat
          # hidden_state = torch.cat((hidden_state[0], hidden_state[1]), -1).unsqueeze(0)
          # cell_state = torch.cat((cell_state[0], cell_state[1]), -1).unsqueeze(0)
          # print('I am here: ', hidden_state.shape)

        #1, 1, 512
        hidden_state = hidden_state.squeeze(0)
        cell_state = cell_state.squeeze(0)
        
        #Feeding <bos> to first lstm
        bos = torch.zeros(images.shape[0]).to(device)
        bos[:] = 117
        bos = torch.nn.functional.one_hot(bos.to(torch.int64), num_classes=vocab_size)

        outputs = []
        #decoder for test
        if object1 is None:
          for t in range(3):
            hidden_state, cell_state = self.lstm_cell(bos.float(), (hidden_state, cell_state))
            hidden_state = self.dropout2(hidden_state)
            cell_state = self.dropout2(cell_state)
            output = self.linear(hidden_state)
            indices = torch.argmax(output, dim=-1)
            bos = torch.nn.functional.one_hot(indices.to(torch.int64), num_classes=vocab_size)
            outputs.append(output)
          return  outputs[0], outputs[1], outputs[2] 

        #decoder for train
        for t in range(3):
          hidden_state, cell_state = self.lstm_cell(bos.float(), (hidden_state, cell_state))
          hidden_state = self.dropout2(hidden_state)
          cell_state = self.dropout2(cell_state)
          output = self.linear(hidden_state)
          if t==1:
            indices = object1
            bos = torch.nn.functional.one_hot(indices.to(torch.int64), num_classes=vocab_size)
          elif t==2:
            indices = relation
            bos = torch.nn.functional.one_hot(indices.to(torch.int64), num_classes=vocab_size)
          outputs.append(output)  
        return outputs[0], outputs[1], outputs[2]

In [ ]:
class Optical_Model(nn.Module):
    def __init__(self, feature_dim=2048, hidden_size=512):
        self.feature_dim = feature_dim
        self.hidden_size = hidden_size
        super(Optical_Model,self).__init__()
        self.lstm = nn.LSTM(input_size=self.feature_dim,
                            hidden_size=self.hidden_size,  # LSTM hidden units
                            num_layers=1,  # number of LSTM layer
                            bias=True,  # use bias weights b_ih and b_hh
                            batch_first=True,  # input & output will have batch size as 1st dimension
                            dropout=0,  # Not applying dropout
                            bidirectional=is_bidirectional,  # unidirectional LSTM
                            )
        self.lstm_cell = nn.LSTMCell(input_size=vocab_size,
                            hidden_size=self.hidden_size,  # LSTM hidden units
                            bias=True,  # use bias weights b_ih and b_hh
                            )
        # self.prelu = nn.PReLU()
        # self.dropout = nn.Dropout(0.5)
        # self.dropout2 = nn.Dropout(0.5)
        self.linear = nn.Linear(self.hidden_size, vocab_size)
        # self.ln = nn.LayerNorm(self.hidden_size, elementwise_affine=False)

    def forward(self, images, object1=None, relation=None):
        # print('images shape',images.shape)
        hidden_state = torch.zeros((2, images.shape[0], self.hidden_size)).cuda()
        cell_state = torch.zeros((2, images.shape[0], self.hidden_size)).cuda()
        
        features = images.squeeze(-1).squeeze(-1) #feature shape: batch, 30, 2048, 1, 1
        # features = self.prelu(features)
        output, (hidden_state, cell_state) = self.lstm(features, (hidden_state, cell_state))
        # output, (hidden_state, cell_state) = self.lstm(features)

        #2, 1, 512 (num of layers * num of directions, batch, dimension)
        print(hidden_state.shape)


        # hidden_state = self.dropout(hidden_state)
        # cell_state = self.dropout(cell_state)
        
        if is_bidirectional:
          # Sum
          hidden_state = torch.sum(torch.stack([hidden_state[0], hidden_state[1]]), 0).unsqueeze(0)
          cell_state = torch.sum(torch.stack([cell_state[0], cell_state[1]]), 0).unsqueeze(0)

          # Concat
          # hidden_state = torch.cat((hidden_state[0], hidden_state[1]), -1).unsqueeze(0)
          # cell_state = torch.cat((cell_state[0], cell_state[1]), -1).unsqueeze(0)
          # print('I am here: ', hidden_state.shape)

        #1, 1, 512
        hidden_state = hidden_state.squeeze(0)
        cell_state = cell_state.squeeze(0)
        
        #Feeding <bos> to first lstm
        bos = torch.zeros(images.shape[0]).to(device)
        bos[:] = 117
        bos = torch.nn.functional.one_hot(bos.to(torch.int64), num_classes=vocab_size)

        outputs = []
        #decoder for test
        if object1 is None:
          for t in range(3):
            hidden_state, cell_state = self.lstm_cell(bos.float(), (hidden_state, cell_state))
            # hidden_state = self.dropout2(hidden_state)
            # cell_state = self.dropout2(cell_state)
            output = self.linear(hidden_state)
            indices = torch.argmax(output, dim=-1)
            bos = torch.nn.functional.one_hot(indices.to(torch.int64), num_classes=vocab_size)
            outputs.append(output)
          return  outputs[0], outputs[1], outputs[2] 

        #decoder for train
        for t in range(3):
          hidden_state, cell_state = self.lstm_cell(bos.float(), (hidden_state, cell_state))
          # hidden_state = self.dropout2(hidden_state)
          # cell_state = self.dropout2(cell_state)
          output = self.linear(hidden_state)
          if t==1:
            indices = object1
            bos = torch.nn.functional.one_hot(indices.to(torch.int64), num_classes=vocab_size)
          elif t==2:
            indices = relation
            bos = torch.nn.functional.one_hot(indices.to(torch.int64), num_classes=vocab_size)
          outputs.append(output)  
        return outputs[0], outputs[1], outputs[2]

#Evaluate

In [ ]:
def Eval(model, validloader, criterion):
  valid_loss,vloss = 0.0, 0.0

  model.eval()
  for step_i,batch in enumerate(iter(validloader)):
      images, object1, relation, object2 = batch[0], batch[1], batch[2], batch[3]
      images = images.to(device)
      object1 = object1.to(device)
      object2 = object2.to(device)
      relation = relation.to(device)
      with torch.no_grad():
          outputs1, outputs2, outputs3 = model(images)  
          print('valid object1', torch.topk(outputs1, 5, dim=-1).indices[:5], 'target', object1[:5])
          print('valid relation2', torch.topk(outputs2, 5, dim=-1).indices[:5], 'target', relation[:5])
          print('valid object23', torch.topk(outputs3, 5, dim=-1).indices[:5], 'target', object2[:5])
          
          output = torch.cat((outputs1, outputs2, outputs3), dim=0)
          target = torch.cat((object1, relation, object2), dim=0)

          valid_loss = criterion(output, target.view(-1))
      
      vloss+=valid_loss

  vloss= vloss / len(validloader)
  # Get Validation statistics.
  stats = 'Validation Loss: %.4f, Perplexity: %5.4f' % (valid_loss.item(), np.exp(valid_loss.item()))
  original_stdout = sys.stdout
  print('\r' + stats)    
  return vloss

#Training

In [ ]:
def Train(model, trainloader, validloader, num_epochs, criterion, optimizer, scheduler, params):
    SEED = 1234

    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed(SEED)

    # Move models to GPU if CUDA is available.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    min_valid_loss = np.inf
    all_epochs_train_loss=[]
    all_epochs_val_loss=[]
    xloss=0
    for epoch in range(1, num_epochs + 1):
        total_step = len(trainloader)

        model.train()
        for step_i, batch in enumerate(iter(trainloader)):

            # Obtain the batch.
            images, object1, relation, object2 = batch[0], batch[1], batch[2], batch[3]

            # Move batch of images and captions to GPU if CUDA is available.
            images = images.to(device)
            object1 = torch.tensor(object1).to(device)
            object2 = torch.tensor(object2).to(device)
            relation = torch.tensor(relation).to(device)

            optimizer.zero_grad()

            outputs1, outputs2, outputs3 = model(images, object1, relation) 
            print('output1', torch.topk(outputs1, 5, dim=-1).indices[:5], 'target', object1[:5])
            print('output2', torch.topk(outputs2, 5, dim=-1).indices[:5], 'target', relation[:5])
            print('output3', torch.topk(outputs3, 5, dim=-1).indices[:5], 'target', object2[:5])

            output = torch.cat((outputs1, outputs2, outputs3), dim=0)
            target = torch.cat((object1, relation, object2), dim=0)

            loss = criterion(output, target.view(-1))
            xloss+=loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(params, 0.1)

            # Update the parameters in the optimizer.
            optimizer.step()


            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (
                epoch, num_epochs, step_i, total_step, loss.item(), np.exp(loss.item()))
            all_epochs_train_loss.append(loss)
            print('\r' + stats)

        vloss = Eval(model, validloader, criterion)
        all_epochs_val_loss.append(vloss)

    
        scheduler.step()
        if min_valid_loss > vloss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{vloss:.6f}) \t Saving The Model')
            # Saving State Dict
            torch.save(model.state_dict(), model_save_path)
            with open(log_path,'a') as lf:
              lf.write(f'\nTraining loss: {loss:.6f} \t Validation Loss Decreased({min_valid_loss:.6f}--->{vloss:.6f}) \t Saving The Model at epoch: {epoch}')
            min_valid_loss = vloss

    return all_epochs_train_loss, all_epochs_val_loss

#Test

In [ ]:
def Test(model):
  SEED = 1234

  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.cuda.manual_seed(SEED)

  # Move models to GPU if CUDA is available.
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  test_dataset = VideoDataset(mode='test')

  testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False)


  model.eval()
  model=model.to(device)
  output_as_str = []

  for images in testloader:
    images = images.to(device)
    with torch.no_grad():
        outputs1, outputs2, outputs3 = model(images)
        
    outputs1[:,35:] = np.inf * -1
    outputs3[:,35:] = np.inf * -1

    outputs2[:,:35] = np.inf * -1
    outputs2[:,-1] = np.inf * -1

    outputs1 = torch.topk(outputs1, 5, dim=-1).indices
    outputs2 = torch.topk(outputs2, 5, dim=-1).indices
    outputs3 = torch.topk(outputs3, 5, dim=-1).indices

    outputs1 = outputs1.cpu()
    outputs2 = outputs2.cpu()
    outputs3 = outputs3.cpu()

    outputs1.apply_(map_to_original_object)
    outputs2.apply_(map_to_original_relation)
    outputs3.apply_(map_to_original_object)

    for i in range(outputs1.shape[0]):
        output_as_str.append(np.array2string(outputs1.cpu().numpy()).replace('[','').replace(']',''))
        output_as_str.append(np.array2string(outputs2.cpu().numpy()).replace('[','').replace(']',''))
        output_as_str.append(np.array2string(outputs3.cpu().numpy()).replace('[','').replace(']',''))

  output_df = pd.DataFrame(output_as_str, columns=['label'])
  output_df.insert(loc=0, column='Id', value=output_df.index)
  return output_df

#Test Ensemble Model

In [ ]:
def Test_Ensemble(model1, model2):
  SEED = 1234

  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.cuda.manual_seed(SEED)

  # Move models to GPU if CUDA is available.
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  test_dataset = VideoDataset(mode='ensemble')

  testloader = torch.utils.data.DataLoader(test_dataset, batch_size=1,shuffle=False)

  model1.eval()
  model2.eval()
  model1=model1.to(device)
  model2=model2.to(device)
  output_as_str = []

  for images, optical_images in testloader:
    images = images.to(device)
    optical_images = optical_images.to(device)
    with torch.no_grad():
        outputs1_1, outputs2_1, outputs3_1 = model1(images)
        outputs1_2, outputs2_2, outputs3_2 = model2(optical_images)

    print(outputs1_1.shape)
    print(outputs1_2.shape)
    print(torch.stack([outputs1_1, outputs1_2]).shape)

    #outputs1 = torch.mean(torch.stack([outputs1_1, outputs1_2]), 0)
    
    outputs1 = outputs1_1 
    # outputs2 = 0.85*outputs2_1 + 0.15*outputs2_2
    outputs3 = outputs3_1
    print(outputs1.shape)

    outputs2 = torch.mean(torch.stack([outputs2_1, outputs2_2]), 0)
    #outputs3 = torch.mean(torch.stack([outputs3_1, outputs3_2]), 0)

    outputs1[:,35:] = np.inf * -1
    outputs3[:,35:] = np.inf * -1

    outputs2[:,:35] = np.inf * -1
    outputs2[:,-1] = np.inf * -1

    outputs1 = torch.topk(outputs1, 5, dim=-1).indices
    outputs2 = torch.topk(outputs2, 5, dim=-1).indices
    outputs3 = torch.topk(outputs3, 5, dim=-1).indices

    outputs1 = outputs1.cpu()
    outputs2 = outputs2.cpu()
    outputs3 = outputs3.cpu()

    outputs1.apply_(map_to_original_object)
    outputs2.apply_(map_to_original_relation)
    outputs3.apply_(map_to_original_object)

    for i in range(outputs1.shape[0]):
        output_as_str.append(np.array2string(outputs1.cpu().numpy()).replace('[','').replace(']',''))
        output_as_str.append(np.array2string(outputs2.cpu().numpy()).replace('[','').replace(']',''))
        output_as_str.append(np.array2string(outputs3.cpu().numpy()).replace('[','').replace(']',''))

  output_df = pd.DataFrame(output_as_str, columns=['label'])
  output_df.insert(loc=0, column='Id', value=output_df.index)
  return output_df

#Main

Create Dataset

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.cuda.manual_seed(SEED)

dataset = VideoDataset()
shuffle_dataset = True

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset:
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validloader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=valid_sampler)

Define Model Params

In [ ]:
# Initialize the model.
model = Model().to(device)

# Define the loss function.
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# Specify the learnable parameters of the model.
params = list(list(model.linear.parameters()) + list(model.lstm.parameters()) + list(model.lstm_cell.parameters()))

# Define the optimizer and scheduler.
optimizer = torch.optim.AdamW(params, lr=1e-03, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

Train Model

In [ ]:
model_save_path = 'models/model.pth'
log_path ='logs/model.txt'

In [ ]:
all_epochs_train_loss, all_epochs_val_loss = Train(model, trainloader, validloader, num_epochs, criterion, optimizer, scheduler, params)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Streaming output truncated to the last 5000 lines.
Epoch [47/50], Step [24/55], Loss: 0.8371, Perplexity: 2.3097
torch.Size([2, 8, 512])
output1 tensor([[29, 19, 10, 30,  8],
        [26, 11,  7, 30, 22],
        [ 7, 26, 11, 20, 16],
        [ 3, 15,  1,  9,  6],
        [16, 17,  0, 31, 13]], device='cuda:0') target tensor([29, 26,  7,  3, 16], device='cuda:0')
output2 tensor([[ 75,  44,  43,  37,  81],
        [ 65,  46, 115,  26, 104],
        [ 35,  65,  93, 112, 115],
        [ 82,  98, 100, 110,  73],
        [ 98, 100,  73,  38,  52]], device='cuda:0') target tensor([37, 49, 93, 82, 70], device='cuda:0')
output3 tensor([[30, 28, 33,  8,  4],
        [26, 11,  7, 22, 30],
        [ 7, 11, 26, 20, 23],
        [ 3,  1, 15, 34,  6],
        [16,  1, 34, 12, 24]], device='cuda:0') target tensor([30, 26,  7,  3, 16], device='cuda:0')
Epoch [47/50], Step [25/55], Loss: 1.4798, Perplexity: 4.3922
torch.Size([2, 8, 512])
output1 tensor([[ 29,   0,  13,   8,  17],
        [ 23,  20,   8

Train optical model

In [ ]:
# Initialize the model.
model = Optical_Model().to(device)

# Define the loss function.
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# Specify the learnable parameters of the model.
params = list(list(model.linear.parameters()) + list(model.lstm.parameters()) + list(model.lstm_cell.parameters()))

# Define the optimizer and scheduler.
optimizer = torch.optim.AdamW(params, lr=1e-03, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
model_save_path = 'models/optical_flow_model.pth'
log_path ='logs/optical_flow_model.txt'

In [ ]:
all_epochs_train_loss, all_epochs_val_loss = Train(model, trainloader, validloader, num_epochs, criterion, optimizer, scheduler, params)

Testing

In [ ]:
optical_model = Model()
model = Model()
optical_model.load_state_dict(torch.load('models/optical_flow_model.pth'))
model.load_state_dict(torch.load('models/model.pth'))

<All keys matched successfully>

In [ ]:
output_df = Test_Ensemble(model, optical_model)

torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([2, 1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
torch.Size([1, 118])
t

In [ ]:
output_df.head()

,Id,label
0,0,29 2 16 3 21
1,1,47 79 28 40 10
2,2,4 30 28 0 17
3,3,29 1 3 6 31
4,4,28 9 47 63 8


In [ ]:
output_csv_path = 'Test_outputs/ensemble_model.csv'

In [ ]:
output_df.to_csv(output_csv_path)